# Autoencoder

In [1]:
from keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, Lambda, Activation, BatchNormalization, LeakyReLU, Dropout
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import plot_model
from keras.datasets import mnist

# GPU related import options - for watching real-time memory use (in console): watch nvidia-smi
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)

import numpy as np
import os

Using TensorFlow backend.


## Load the data

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.astype('float32') / 255.
x_test = x_test.reshape(x_test.shape + (1,))

## Define the structure of the neural network

In [3]:
input_dim = (28, 28, 1)
encoder_conv_filters = [32,64,64,64]
encoder_conv_kernel_size = [3,3,3,3]
encoder_conv_strides = [1,2,2,1]
decoder_conv_t_filters = [64,64,32,1]
decoder_conv_t_kernel_size = [3,3,3,3]
decoder_conv_t_strides = [1,2,2,1]
z_dim = 2

use_batch_norm = False
use_dropout = False

n_layers_encoder = len(encoder_conv_filters)
n_layers_decoder = len(decoder_conv_t_filters)

## Encoder

Define the input to the encoder (the image)

In [4]:
encoder_input = Input(shape = input_dim, name = 'encoder_input')

In [5]:
x = encoder_input

for i in range(n_layers_encoder):
    conv_layer = Conv2D(
        filters = encoder_conv_filters[i],
        kernel_size = encoder_conv_kernel_size[i],
        strides = encoder_conv_strides[i],
        padding = 'same',
        name = 'encoder_conv_' + str(i))
    
    x = conv_layer(x) # Stack the convolutional layers on top of each other
    x = LeakyReLU()(x)
    
    if use_batch_norm:
        x = BatchNormalization()(x)
    if use_dropout:
        x = Dropout(rate = 0.25)(x)

shape_before_flattening = K.int_shape(x)[1:]

x = Flatten()(x) # Flatten the last conv. layer to a vector

Dense layer that connects the vector to the 2D latent space

In [6]:
encoder_output = Dense(z_dim, name = 'encoder_output')(x)

Keras model defining the encoder - a model that takes an input image and encodes it into the 2D latent space

In [7]:
encoder = Model(encoder_input, encoder_output)

In [8]:
encoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 28, 28, 1)         0         
_________________________________________________________________
encoder_conv_0 (Conv2D)      (None, 28, 28, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 28, 28, 32)        0         
_________________________________________________________________
encoder_conv_1 (Conv2D)      (None, 14, 14, 64)        18496     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
encoder_conv_2 (Conv2D)      (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 7, 7, 64)          0   

## Decoder

Defines the input to the decoder (the point in the latent space)

In [9]:
decoder_input = Input(shape = (z_dim, ), name = 'decoder_input')

Connect the input to a Dense layer

In [10]:
x = Dense(np.prod(shape_before_flattening))(decoder_input)

Reshape the vector into a tensor that can be fed as input into the first conv. transpose layer

In [11]:
x = Reshape(shape_before_flattening)(x)

In [12]:
for i in range(n_layers_decoder):
    conv_t_layer = Conv2DTranspose(
        filters = decoder_conv_t_filters[i],
        kernel_size = decoder_conv_t_kernel_size[i],
        strides = decoder_conv_t_strides[i],
        padding = 'same',
        name = 'decoder_conv_t_' + str(i))
    
    x = conv_t_layer(x) # Stack the conv. transpose layers on top of each other
    
    if i < n_layers_decoder - 1:
        x = LeakyReLU()(x)
        
        if use_batch_norm:
            x = BatchNormalization()(x)
        if use_dropout:
            x = Dropout(rate = 0.25)(x)
    else:
        x = Activation('sigmoid')(x)

In [13]:
decoder_output = x

Keras model defining the decoder - a model that takes a point in the latent space and decodes it into the original image domain

In [14]:
decoder = Model(decoder_input, decoder_output)

In [15]:
decoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 2)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 3136)              9408      
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_1 (Conv2DTran (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 14, 14, 64)        0   

## Joining the Encoder to the Decoder

In [16]:
model_input = encoder_input # The input to the autoencoder is the same as the input to the encoder
model_output = decoder(encoder_output) # The output from the autoencoder is the output from the encoder passed through the decoder

The Keras model defining the full autoencoder - a model taking a image, passing it through the encoder and back out through the decoder to generate a reconstruction of the original image

In [17]:
model = Model(model_input, model_output)

### Compilation

In [18]:
learning_rate = 0.0005
batch_size = 32
initial_epoch = 0

In [19]:
optimizer = Adam(lr = learning_rate)

def r_loss(y_true, y_pred):
    return K.mean(K.square(y_true - y_pred), axis = [1, 2, 3])

model.compile(optimizer = optimizer, loss = r_loss)

### Training

In [ ]:
model.fit(x = x_train,
          y = x_train,
          batch_size = batch_size,
          shuffle = True,
          epochs = 200,
          initial_epoch = initial_epoch)

Epoch 1/200
60000/60000 [==============================] - 13s 210us/step - loss: 0.0551
Epoch 2/200
60000/60000 [==============================] - 11s 182us/step - loss: 0.0462
Epoch 3/200
60000/60000 [==============================] - 11s 188us/step - loss: 0.0442
Epoch 4/200
60000/60000 [==============================] - 12s 197us/step - loss: 0.0431
Epoch 5/200
60000/60000 [==============================] - 12s 194us/step - loss: 0.0424
Epoch 6/200
60000/60000 [==============================] - 11s 189us/step - loss: 0.0419
Epoch 7/200
60000/60000 [==============================] - 11s 184us/step - loss: 0.0415
Epoch 8/200
60000/60000 [==============================] - 10s 175us/step - loss: 0.0412
Epoch 9/200
60000/60000 [==============================] - 11s 185us/step - loss: 0.0409
Epoch 10/200
60000/60000 [==============================] - 12s 196us/step - loss: 0.0407
Epoch 11/200
60000/60000 [==============================] - 11s 190us/step - loss: 0.0405
Epoch 12/200
60000/